###Getting Movie Franchise Information

In [74]:
import requests
from bs4 import BeautifulSoup
import pickle
import time
from collections import defaultdict

In [2]:
import os
os.getcwd()
os.chdir('/Users/ItelinaMa/Documents/Metis/Luther')

In [110]:
urls = ['http://www.boxofficemojo.com/franchises/?view=Brand&p=.htm', 'http://www.boxofficemojo.com/franchises/?view=Franchise&p=.htm']

In [111]:
def getSoups(urls):
    soups = []
    for url in urls:
        response = requests.get(url)
        soup = BeautifulSoup(response.text)
        soups.append(soup)
    return soups

In [113]:
soups = getSoups(urls)

#####Get brands and corresponding links

In [138]:
def getLinks(soup):
    links =[]
    brands = []
    for i in range(2, len(soup.find_all('table')[2].find_all('tr'))):
        link = soup.find_all('table')[2].find_all('tr')[i].find_all('a')[0]['href']
        newlink = 'http://www.boxofficemojo.com/franchises' + link.replace('.', '', 1)
        links.append(newlink)
        brands.append(soup.find_all('table')[2].find_all('tr')[i].find_all('a')[0].text)
    return links, brands

In [140]:
linksB, brands = getLinks(soups[0])
linksB[-1], brands[-1]

('http://www.boxofficemojo.com/franchises/chart/?id=disneyanimation.htm',
 u'Walt Disney Animation Studios')

In [141]:
linksS, series = getLinks(soups[1])
linksS[-1], series[-1]

('http://www.boxofficemojo.com/franchises/chart/?id=xmen.htm', u'X-Men')

In [142]:
def getFranchise(franchise=brands, links=linksB):
    Franchise = defaultdict(list)
    for i, item in enumerate(franchise):
        url = links[i]
        response = requests.get(url)
        while response.status_code != 200:
            time.sleep(2)
            response = requests.get(url)
        soup = BeautifulSoup(response.text)
        movies = []
        for i in range(1, len(soup.find_all('table')[3].find_all('table')[0].find_all('tr'))-2):
            movies.append(soup.find_all('table')[3].find_all('table')[0].find_all('tr')[i].find_all('td')[1].text)
        Franchise[item] = movies
    return Franchise

In [143]:
franchiseBrands = getFranchise(brands,linksB)

In [145]:
len(franchiseBrands)

33

In [144]:
franchiseSeries = getFranchise(series,linksS)

In [146]:
len(franchiseSeries)

153

In [147]:
with open('franchiseBrands.pkl', 'w') as picklefile:
    pickle.dump(franchiseBrands, picklefile)

In [148]:
with open('franchiseSeries.pkl', 'w') as picklefile:
    pickle.dump(franchiseSeries, picklefile)